Какие товарные подгруппы дают значимое увеличение показателей при проведении акций по скидкам. Рассчитать коэффициенты прироста / спада показателей для каждой товарной подгруппы, для каждого товара. 

Оценить достоверность оценки (достаточно будет рассчитать кол-во событий, на которых проводится анализ – чтобы отфильтровать по значимому кол-ву событий). Примечание: событие это чек. Для оценки значимости нужно знать кол-во чеков, по которым проводился расчет. 

Пример: если товар А был в одном чеке за период без акций, а в периоде акций в двух чеках, то рост в два раза малозначим. Для такой оценки нужно знать сколько чеков было в расчете с «обоих сторон» сравнения. При усреднении товаров в группу учесть можно будет в качестве веса примера по товару. Например, по товару А всего 3 чека, а по товару Б той же группы 100 чеков, но рост 1.1. Усреднять можно как = (2 *3 + 1.1 *100) / 103.

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.precision", 3)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_rows", 25)

In [3]:
df_chech = pd.read_csv('data/row/cheques_rows.csv', sep=';')

In [4]:
df_prod = pd.read_csv('data/row/product_df.csv', sep=";")

In [5]:
df = pd.merge(df_chech, df_prod, how='left', left_on='product_id', right_on='product_id')

In [6]:
df['sale_date_id'] =  pd.to_datetime(df['sale_date_id'], format='%Y%m%d')

In [16]:
df.sample()

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id
13377657,403419,2020-05-26,172439,33,129257,1.0,28.99,34.099,1392


In [14]:
df.to_csv('data/interim/df.csv')

TypeError: to_csv() got an unexpected keyword argument 'index_col'

In [3]:
df = pd.read_csv('data/interim/df.csv', index_col=0)

/home/egor/anaconda3/envs/BlCh_3.8.2/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df.head()

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id
0,25,2020-05-02,25,22,1396,1.0,278.99,0.0,26
1,25,2020-05-02,25,22,91519,2.0,139.78,0.0,944
2,25,2020-05-02,25,22,106983,2.0,87.98,0.0,1138
3,25,2020-05-02,25,22,106746,1.0,55.99,0.0,1134
4,25,2020-05-02,25,22,106676,1.0,59.99,0.0,1129


In [109]:
count_days = df.agg({'sale_date_id' : 'nunique'})

In [110]:
count_days

sale_date_id    28
dtype: int64

In [7]:
df['sale'] = np.where(df['discount_prc'] >  (df['sum_discounted'] + df['discount_prc'] )* 0.05, 'Sale', 'Reg')

In [34]:
df.sort_values('sale_date_id')

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,total_disc,sale
0,25,2020-05-02,25,22,1396,1.000,278.99,0.000,26,0.000,Reg
376534,1064672,2020-05-02,58651,12,22048,1.000,49.99,0.000,327,0.000,Reg
376533,1064672,2020-05-02,58651,12,107071,1.000,57.99,0.000,1140,0.000,Reg
376532,1064672,2020-05-02,58651,12,64694,2.000,9.98,0.000,851,0.000,Reg
376531,1064672,2020-05-02,58651,12,106656,1.000,39.99,0.000,1129,0.000,Reg
...,...,...,...,...,...,...,...,...,...,...,...
15104769,472249,2020-05-29,96785,65,112889,2.000,71.98,10.003,1192,20.006,Sale
15104770,472249,2020-05-29,96785,65,100412,1.000,59.97,36.867,1087,36.867,Sale
15104771,472249,2020-05-29,96785,65,106756,1.000,42.99,14.003,1134,14.003,Sale
15104764,472249,2020-05-29,96785,65,64813,1.000,4.99,0.000,851,0.000,Reg


TypeError: unsupported operand type(s) for -: 'str' and 'str'

### Проверка на наличие неодназначных продаж. Таких продаж, которые проходили в один день, в одной подгруппе товаров, в разных чеках, но с разными условиями - по акции и нет 

In [5]:
df['status_g'] = np.where(
    (df['sale_date_id'] == df['sale_date_id']) &
    (df['product_sub_group_id'] == df['product_sub_group_id'])&
    (df['cheque_id'] != df['cheque_id'])  &
    (df['sale'] != df['sale'])
    ,'Bad', 'Good'
)

In [6]:
df.loc[df['status_g'] == 'Bad']

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g


### На всякий случай, делаю такую же проверку для всех товаров

In [7]:
df['status_p'] = np.where(
    (df['sale_date_id'] == df['sale_date_id']) &
    (df['product_id'] == df['product_id'])&
    (df['cheque_id'] != df['cheque_id'])  &
    (df['sale'] != df['sale'])
    ,'Bad', 'Good'
)

In [8]:
df.loc[df['status_p'] == 'Bad']

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g,status_p


## По результатам вычислений, неодназначных продаж не обнаруженно 

In [9]:
df.to_csv('data/interim/df_with_status.csv')

In [4]:
df = pd.read_csv('data/interim/df_with_status.csv', index_col=0)

In [5]:
df.head()

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g,status_p
0,25,2020-05-02,25,22,1396,1.0,278.99,0.0,26,Reg,Good,Good
1,25,2020-05-02,25,22,91519,2.0,139.78,0.0,944,Reg,Good,Good
2,25,2020-05-02,25,22,106983,2.0,87.98,0.0,1138,Reg,Good,Good
3,25,2020-05-02,25,22,106746,1.0,55.99,0.0,1134,Reg,Good,Good
4,25,2020-05-02,25,22,106676,1.0,59.99,0.0,1129,Reg,Good,Good


In [6]:
df_checks_by_status = df.groupby(by=['sale', 'product_sub_group_id'], as_index = False).agg({'cheque_id' : pd.Series.nunique})

In [9]:
df_checks_by_status.head()

,sale,product_sub_group_id,cheque_id
0,Reg,0,4238
1,Reg,1,11716
2,Reg,2,4849
3,Reg,3,11688
4,Reg,4,1479


In [8]:
df_days = df.groupby(by=[ 'product_sub_group_id' , 'sale', 'shop_id']).agg({'sale_date_id': pd.Series.nunique })

In [32]:
df_days.to_csv('data/interim/df_days.csv')

In [107]:
df_days

sale_date_id
product_sub_group_id sale shop_id              
0                    Reg  1                  15
                          2                   7
                          3                  21
                          4                  16
                          5                  15
...                                         ...
1427                 Sale 43                  2
                          55                  1
                          69                  4
                          70                  2
                          75                  1

[103786 rows x 1 columns]

In [66]:
df_days_wtout_shop = df.groupby(by=['sale' ,'product_sub_group_id'], as_index = False).agg({'sale_date_id': pd.Series.nunique })

In [67]:
df_days_wtout_shop.head()

,sale,product_sub_group_id,sale_date_id
0,Reg,0,28
1,Reg,1,28
2,Reg,2,28
3,Reg,3,28
4,Reg,4,28


In [27]:
df_reg = df.loc[df['sale'] == 'Reg'].groupby('product_id').agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_reg', 
    'quantity': 'quantity_reg', 
    'sum_discounted': 'sum_reg'}, axis='columns')

In [28]:
df_reg.head()

,cheque_count_reg,quantity_reg,sum_reg
product_id,,,
0,3961,6448.556,428117.07
101,276,298.000,30188.14
102,1,1.000,69.91
107,185,218.000,56403.44
108,475,532.000,89650.12


In [76]:
df_sales = df.loc[df['sale'] == 'Sale'].groupby('product_id').agg({
    'cheque_id' : 'count',
    'quantity' : 'sum', 
    'sum_discounted': 'sum'
}).rename({
    'cheque_id': 'cheque_count_sales', 
    'quantity': 'quantity_sales', 
    'sum_discounted': 'sum_sales'}, axis='columns')

In [77]:
df_sum = pd.concat([df_reg, df_sales], axis=1)

In [78]:
df_sum

,cheque_count_reg,quantity_reg,sum_reg,cheque_count_sales,quantity_sales,sum_sales
product_id,,,,,,
0,3961.0,6448.556,428117.07,1087.0,1440.893,64047.01
101,276.0,298.000,30188.14,31.0,32.000,2547.51
102,1.0,1.000,69.91,NaN,NaN,NaN
107,185.0,218.000,56403.44,57.0,71.000,10786.95
108,475.0,532.000,89650.12,193.0,240.000,24398.87
...,...,...,...,...,...,...
130662,NaN,NaN,NaN,2.0,0.000,0.00
130671,1.0,1.000,6398.40,NaN,NaN,NaN
130676,1.0,1.000,86677.60,NaN,NaN,NaN


In [4]:
def coefficent(x, y):
    return (y/x)*100

In [80]:
df_sum.to_csv('data/interim/df_sum_item.csv')

In [1]:
df_sum = pd.read_csv('data/interim/df_sum.csv', index_col=0)

NameError: name 'pd' is not defined

In [31]:
df_sum.head()

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,status_g,status_p
0,25,2020-05-02,25,22,1396,1.0,278.99,0.0,26,Reg,Good,Good
1,25,2020-05-02,25,22,91519,2.0,139.78,0.0,944,Reg,Good,Good
2,25,2020-05-02,25,22,106983,2.0,87.98,0.0,1138,Reg,Good,Good
3,25,2020-05-02,25,22,106746,1.0,55.99,0.0,1134,Reg,Good,Good
4,25,2020-05-02,25,22,106676,1.0,59.99,0.0,1129,Reg,Good,Good


In [ ]:
df_sum.groupby(by=['product_sub_group_id ,sale_date_id'])

In [7]:
df_sum['c_check'] = coefficent(df_sum['cheque_count_reg'],df_sum['cheque_count_sales'])

In [8]:
df_sum['c_qua'] = coefficent(df_sum['quantity_reg'],df_sum['quantity_sales'])
df_sum['c_sum'] = coefficent(df_sum['sum_reg'],df_sum['sum_sales'])

In [9]:
df_sum.loc[df_sum['c_sum'] > 0]

,cheque_count_reg,quantity_reg,sum_reg,cheque_count_sales,quantity_sales,sum_sales,c_check,c_qua,c_sum
product_sub_group_id,,,,,,,,,
0,4238.0,6747.556,4.584e+05,1118.0,1472.893,6.659e+04,26.380,21.829,14.528
1,11729.0,22980.000,2.402e+06,17810.0,18086.000,1.795e+06,151.846,78.703,74.736
2,4915.0,7048.429,5.886e+05,4659.0,6549.646,5.130e+05,94.791,92.923,87.153
3,11952.0,17470.572,2.617e+06,7530.0,10832.847,1.174e+06,63.002,62.006,44.861
4,1491.0,2740.207,6.166e+05,1577.0,2361.919,3.144e+05,105.768,86.195,50.995
...,...,...,...,...,...,...,...,...,...
1417,12804.0,14546.000,2.313e+06,14825.0,17243.000,1.689e+06,115.784,118.541,73.002
1418,249.0,67.954,2.549e+04,18.0,5.795,1.751e+03,7.229,8.528,6.868
1419,22.0,23.660,1.232e+04,5.0,4.106,1.620e+03,22.727,17.354,13.144


Почистим от тех товаров, которые продавались только по акции или только по обычным ценам. 

In [10]:
df_sum.dropna(inplace=True)

In [12]:
df_sum = pd.merge(df_sum, df_prod, how='left', left_on='product_id', right_on='product_id')

KeyError: 'product_id'

In [13]:
df_sum.head()

,cheque_count_reg,quantity_reg,sum_reg,cheque_count_sales,quantity_sales,sum_sales,c_check,c_qua,c_sum
product_sub_group_id,,,,,,,,,
0,4238.0,6747.556,4.584e+05,1118.0,1472.893,6.659e+04,26.380,21.829,14.528
1,11729.0,22980.000,2.402e+06,17810.0,18086.000,1.795e+06,151.846,78.703,74.736
2,4915.0,7048.429,5.886e+05,4659.0,6549.646,5.130e+05,94.791,92.923,87.153
3,11952.0,17470.572,2.617e+06,7530.0,10832.847,1.174e+06,63.002,62.006,44.861
4,1491.0,2740.207,6.166e+05,1577.0,2361.919,3.144e+05,105.768,86.195,50.995


In [14]:
df_sum['r_check'] = (df_sum['cheque_count_sales'] * df_sum['c_check']) 

In [15]:
def r_coef(df_sub):
    return df_sub['r_check'].sum() / df_sub['quantity_sales'].sum()

In [16]:
df_sum_group = df_sum.groupby('product_sub_group_id').apply(r_coef)

In [17]:
result = pd.DataFrame({'product_sub_group_coef_sales':df_sum_group.values}, index=df_sum_group.index)

In [18]:
result

,product_sub_group_coef_sales
product_sub_group_id,
0,20.024
1,149.529
2,67.429
3,43.793
4,70.619
...,...
1417,99.548
1418,22.454
1419,27.676


In [19]:
result.to_csv('data/interim/result.csv')

In [72]:
result = pd.read_csv('data/interim/result.csv', index_col=0)

7.Для упрощения расчета период акции определять как: в эту дату в этом магазине по товару не было предоставлено ни одной скидки. Значение самого периода (даты с / по) вычислять не обязательно. Но необходимо рассчитать кол-во дней со скидками / без скидок по анализируемому товару / подгруппе (чтобы не было перекосов по дням).

In [33]:
df_days.head()

sale_date_id
product_sub_group_id sale shop_id              
0                    Reg  1                  15
                          2                   7
                          3                  21
                          4                  16
                          5                  15

In [69]:
df_days_wtout_shop.loc[df_days_wtout_shop['sale'] == 'Sale']

,sale,product_sub_group_id,sale_date_id
998,Sale,0,26
999,Sale,1,28
1000,Sale,2,28
1001,Sale,3,28
1002,Sale,4,28
...,...,...,...
1841,Sale,1417,28
1842,Sale,1418,9
1843,Sale,1419,4
1844,Sale,1421,13


In [102]:
result_2 = pd.merge(result, df_days_wtout_shop.loc[df_days_wtout_shop['sale'] == 'Sale'], how='left', left_on='product_sub_group_id', right_on='product_sub_group_id')

In [78]:
df_reg_days = df_days_wtout_shop.loc[df_days_wtout_shop['sale'] == 'Reg']

In [96]:
df_reg_days = df_reg_days.rename(columns={'sale_date_id': 'reg_day_count', 'sale' : 'reg'})

In [124]:
count_days[:1:]

Series([], dtype: int64)

In [127]:
result_2['reg_date_count'] =  count_days.item() - result_2['sale_date_id']

In [128]:
result_2.head()

,product_sub_group_id,product_sub_group_coef_sales,sale,sale_date_id,reg_date_count
0,0,20.024,Sale,26,2
1,1,149.529,Sale,28,0
2,2,67.429,Sale,28,0
3,3,43.793,Sale,28,0
4,4,70.619,Sale,28,0


In [11]:
result_3 = pd.merge(result_2, df_checks_by_status.loc[df_checks_by_status['sale'] == 'Sale'],  how='left', left_on='product_sub_group_id', right_on='product_sub_group_id')

NameError: name 'result_2' is not defined

# Показатели

In [11]:
df_mean_cheque = df.groupby(by=['sale']).agg({'sum_discounted' : 'sum', 'cheque_id' : 'count' })

In [12]:
df_mean_cheque['mean_cheque'] = df_mean_cheque['sum_discounted'] / df_mean_cheque['cheque_id']

In [13]:
df_mean_cheque.head()

,sum_discounted,cheque_id,mean_cheque
sale,,,
Reg,7.467e+08,8292019,90.055
Sale,6.304e+08,7246258,86.995


Частота чеков = кол-во разных чеков по одному  контакту / продолжительность акционного периода / без акций. Усредняется по всем контактам.

In [14]:
df.head()

,cheque_id,sale_date_id,contact_id,shop_id,product_id,quantity,sum_discounted,discount_prc,product_sub_group_id,sale,mean_cheque
0,25,2020-05-02,25,22,1396,1.0,278.99,0.0,26,Reg,11.160
1,25,2020-05-02,25,22,91519,2.0,139.78,0.0,944,Reg,5.591
2,25,2020-05-02,25,22,106983,2.0,87.98,0.0,1138,Reg,3.519
3,25,2020-05-02,25,22,106746,1.0,55.99,0.0,1134,Reg,2.240
4,25,2020-05-02,25,22,106676,1.0,59.99,0.0,1129,Reg,2.400


In [16]:
df.groupby(by=['sale', 'contact_id', 'shop_id']).agg({'sum_discounted' : 'sum'})

sum_discounted
sale contact_id shop_id                
Reg  1          1               1096.37
                34                78.81
     2          2              15044.25
                32               906.81
     3          3               1732.66
...                                 ...
Sale 267540     65                94.99
     267543     45               272.90
     267545     75               319.90
     267546     32                39.90
     267549     9                159.90

[1030444 rows x 1 columns]

In [ ]:
df.groupby(by=['contact_id']).agg({'sale_date_id': "nunique" })